In [ ]:
import datetime as dt
import functools
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display, Markdown


In [ ]:
# Configuration

raw_data_dir = Path("../../raw_data")
reports_dir = Path("../../reports")
season_id = 25
week_date = dt.datetime(2019, 10, 31)
season_raw_data_dir = raw_data_dir / f"season{season_id}"
season_reports_dir = reports_dir / f"season{season_id}"
season_box_scores_dir = raw_data_dir / f"season{season_id}" / "box_scores"
PLAYERS_MIN_GAMES_PLAYED = 1

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

possessions = pd.read_csv(season_raw_data_dir / "possessions.csv")


In [ ]:
summary_rows = []
for (game_id, team_id), group in possessions.groupby(["game_id", "team_id"]):
    row = {
        "game_id": game_id,
        "team_id": team_id,
        "possessions": group["game_id"].size,
        "players_involved": group["players_involved"].mean(),
        "points_scored": group["points_scored"].mean(),
        "passes": group["passes"].mean(),
        "oreb": group["oreb"].mean(),
        "fd": group["nsfd"].mean() + group["sfd"].mean(),
        "fta": group["fta"].mean(),
    }
    summary_rows.append(row)

summary = pd.DataFrame(summary_rows)
summary
